## HHI Tables

In [1]:
# Import packages
import os
import sys
import pandas as pd
import numpy as np
import logging
import yaml
import message_ix
import ixmp
import itertools

import plotly.graph_objects as go
import bokeh
import holoviews as hv
from holoviews import dim, opts

from message_ix_models.tools.bilateralize.utils import load_config
from message_ix_models.util import package_data_path
from ixmp import Platform

[WARNING] 12:11:09 - pint.util: Redefining 'EUR_2005' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
[WARNING] 12:11:09 - pint.util: Redefining 'EUR' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
[INFO] 12:11:09 - message_ix_models.util.context: Create root Context
[DEBUG] 12:11:09 - genno.config: Override ConfigHandler(key='iamc', callback=<function iamc at 0x000001A78D3F60C0>, iterate=True, discard=True)


In [23]:
# Bring in configuration
config, config_path = load_config(project_name = 'alps_hhi', 
                                  config_name = 'config.yaml')
data_path = os.path.dirname(config_path)

# Import dataframe
df_supply = pd.read_csv(package_data_path('alps_hhi', 'reporting', 'reporting.csv'))
df_supply["importer"] = df_supply["region"]
df_imports = df_supply[df_supply['variable'].str.contains('Domestic') == False]

df_imports[(df_imports['region'] == 'R12_WEU')&(df_imports['year'] == 2100)].to_csv('check.csv')

In [29]:
# Calculate HHI
def calculate_hhi(df, region_list):
    df = df[df['region'].isin(region_list)]
    df_tot = df.groupby(['model', 'scenario', 'unit', 'year'])['value'].sum().reset_index()
    df_tot = df_tot.rename(columns = {'value': 'total_com'})
    df_hhi = df.merge(df_tot, on = ['model', 'scenario', 'unit', 'year'], how = 'left').reset_index()
    df_hhi['HHI'] = (df_hhi['value'] / df_hhi['total_com'])*(df_hhi['value'] / df_hhi['total_com'])
    df_hhi = df_hhi.groupby(['model', 'scenario', 'unit', 'year'])['HHI'].sum().reset_index()

    return df_hhi
    
hhi_supply = calculate_hhi(df_supply, ['R12_WEU', 'R12_EEU'])
hhi_imports = calculate_hhi(df_imports, ['R12_WEU', 'R12_EEU'])

In [33]:
# Display tables
hhidf = hhi_imports.copy()
years = [2025, 2030, 2040, 2050, 2060, 2080, 2100]
regions = ['R12_WEU']
scenarios = ['SSP2', 'SSP2_hhi_HC_imports', 'SSP2_hhi_WS_l90p_imports',
             'SSP2_hhi_HC_supply', 'SSP2_hhi_WS_l90p_supply',
             'SSP2_FSU_EUR_2040', 'SSP2_FSU_EUR_2110',
             'SSP2_hhi_HC_FSU_EUR_2110', 'SSP2_hhi_WS_l90p_supply_FSU_EUR_2110',
             'SSP2_FSU_EUR_NAM_PAO_2110']
             
tdf = hhidf[hhidf['year'].isin(years)]
tdf = tdf[['scenario', 'year', 'HHI']]

tdf_out = pd.DataFrame(columns = ['scenario'])
for y in years:
    tdfy = tdf[tdf['year'] == y]
    tdfy = tdfy[['scenario', 'HHI']]
    tdfy = tdfy.rename(columns = {'HHI': str(y)})
    tdf_out = tdf_out.merge(tdfy, left_on = ['scenario'], right_on = ['scenario'], how = 'outer')

tdf_out
    


,scenario,2025,2030,2040,2050,2060,2080,2100
0,SSP2,0.157040,0.194216,0.293845,0.529793,0.432786,0.208360,0.259753
1,SSP2_FSU_EUR_2040,0.157040,0.260801,0.324967,0.412270,0.428472,0.512038,0.521223
2,SSP2_FSU_EUR_2110,0.157040,0.264805,0.358038,0.439006,0.422173,0.271158,0.259286
3,SSP2_FSU_EUR_NAM_PAO_2110,0.157040,0.212194,0.196231,0.469545,0.465180,0.302680,0.252448
4,SSP2_hhi_HC_imports,0.157040,0.144009,0.137374,0.104995,0.097349,0.084015,0.089552
5,SSP2_hhi_HC_imports_FSU_EUR_2110,0.157040,0.182306,0.165123,0.147519,0.125875,0.102706,0.098038
6,SSP2_hhi_HC_supply,0.157040,0.143984,0.137901,0.105565,0.094611,0.084604,0.092638
7,SSP2_hhi_WS_l90p_imports,0.157040,0.149364,0.129439,0.087529,0.081484,0.067760,0.066020
8,SSP2_hhi_WS_l90p_supply,0.157040,0.150006,0.131058,0.087878,0.081988,0.068047,0.065479
9,SSP2_hhi_WS_l90p_supply_FSU_EUR_2110,0.157040,0.178321,0.170557,0.124040,0.106368,0.083644,0.074810
